# Model

In [4]:
import torch

from torchinfo import summary
from transformers import BatchEncoding, TextStreamer
from unsloth import FastLanguageModel

/tmp/ipykernel_61464/407457176.py:5: UserWarning: WARNING: Unsloth should be imported before [transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Could not import trl.trainer.dpo_trainer: Failed to import trl.trainer.dpo_trainer because of the following error (look up to see its traceback):
cannot import name 'MODEL_FOR_VISION_2_SEQ_MAPPING_NAMES' from 'transformers.models.auto.modeling_auto' (/home/anderson/.pyenv/versions/3.12.10/lib/python3.12/site-packages/transformers/models/auto/modeling_auto.py)


In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gpt-oss-20b",
    dtype=None,  # torch.bfloat16,  # None for auto detection
    max_seq_length=1000,
    load_in_4bit=False,
    full_finetuning=False,
    low_cpu_mem_usage=True,
    device_map="cuda",  # Explicitly load to CUDA
)

[huggingface_hub.utils._http|WARNING]Warning: You are sending unauthenticated requests to the HF Hub. Please set a HF_TOKEN to enable higher rate limits and faster downloads.


==((====))==  Unsloth 2026.1.4: Fast Gpt_Oss patching. Transformers: 5.0.0.
   \\   /|    NVIDIA RTX PRO 6000 Blackwell Workstation Edition. Num GPUs = 1. Max memory: 94.968 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu130. CUDA: 12.0. CUDA Toolkit: 13.0. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


MXFP4 quantization requires Triton and kernels installed: CUDA requires Triton >= 3.4.0, XPU requires Triton >= 3.5.0, we will default to dequantizing the model to bf16


Loading weights:   0%|          | 0/411 [00:00<?, ?it/s]

# Model Summary

In [12]:
import torch
from rich.tree import Tree
from rich import print as rprint

def visualize_model_structure(model):
    # 1. Create root node
    tree = Tree(f"🏗️ [bold blue]Model: {getattr(model.config, '_name_or_path', 'Unknown')}[/bold blue]")
    
    # Dictionary to keep track of created nodes: {path_string: rich_tree_node}
    node_lookup = {"": tree}

    for name, module in model.named_modules():
        if name == "": continue
        
        # Split path: 'model.layers.0.self_attn' -> ['model', 'layers', '0', 'self_attn']
        parts = name.split('.')
        parent_path = ".".join(parts[:-1])
        current_part = parts[-1]

        # Calculate Size Info
        # Get parameter count for this specific module
        params_count = sum(p.numel() for p in module.parameters(recurse=False))
        
        # Get shape if it's a leaf layer (like Linear or Embedding)
        shape_info = ""
        if hasattr(module, 'weight') and isinstance(module.weight, torch.Tensor):
            shape_info = f" [yellow]({list(module.weight.shape)})[/yellow]"
        elif params_count > 0:
            shape_info = f" [green]({params_count:,} params)[/green]"

        # 2. Find or Create Node
        if parent_path in node_lookup:
            parent_node = node_lookup[parent_path]
            # Add new node with style and size info
            new_node = parent_node.add(f"[bold magenta]{current_part}[/bold magenta]{shape_info}")
            node_lookup[name] = new_node

    rprint(tree)

# Execution
visualize_model_structure(model)

🏗️ Model: unsloth/gpt-oss-20b
├── model
│   ├── embed_tokens ([201088, 2880])
│   ├── layers
│   │   ├── 0
│   │   │   ├── self_attn (64 params)
│   │   │   │   ├── q_proj ([4096, 2880])
│   │   │   │   ├── k_proj ([512, 2880])
│   │   │   │   ├── v_proj ([512, 2880])
│   │   │   │   └── o_proj ([2880, 4096])
│   │   │   ├── mlp
│   │   │   │   ├── router ([32, 2880])
│   │   │   │   └── experts (796,538,880 params)
│   │   │   ├── input_layernorm ([2880])
│   │   │   └── post_attention_layernorm ([2880])
│   │   ├── 1
│   │   │   ├── self_attn (64 params)
│   │   │   │   ├── q_proj ([4096, 2880])
│   │   │   │   ├── k_proj ([512, 2880])
│   │   │   │   ├── v_proj ([512, 2880])
│   │   │   │   └── o_proj ([2880, 4096])
│   │   │   ├── mlp
│   │   │   │   ├── router ([32, 2880])
│   │   │   │   └── experts (796,538,880 params)
│   │   │   ├── input_layernorm ([2880])
│   │   │   └── post_attention_layernorm ([2880])
│   │   ├── 2
│   │   │   ├── self_attn (64 params)
│   │   │   │   ├── q_proj ([4096, 2880])
│   │   │   │   ├── k_proj ([512, 2880])
│   │   │   │   ├── v_proj ([512, 2880])
│   │   │   │   └── o_proj ([2880, 4096])
│   │   │   ├── mlp
│   │   │   │   ├── router ([32, 2880])
│   │   │   │   └── experts (796,538,880 params)
│   │   │   ├── input_layernorm ([2880])
│   │   │   └── post_attention_layernorm ([2880])
│   │   ├── 3
│   │   │   ├── self_attn (64 params)
│   │   │   │   ├── q_proj ([4096, 2880])
│   │   │   │   ├── k_proj ([512, 2880])
│   │   │   │   ├── v_proj ([512, 2880])
│   │   │   │   └── o_proj ([2880, 4096])
│   │   │   ├── mlp
│   │   │   │   ├── router ([32, 2880])
│   │   │   │   └── experts (796,538,880 params)
│   │   │   ├── input_layernorm ([2880])
│   │   │   └── post_attention_layernorm ([2880])
│   │   ├── 4
│   │   │   ├── self_attn (64 params)
│   │   │   │   ├── q_proj ([4096, 2880])
│   │   │   │   ├── k_proj ([512, 2880])
│   │   │   │   ├── v_proj ([512, 2880])
│   │   │   │   └── o_proj ([2880, 4096])
│   │   │   ├── mlp
│   │   │   │   ├── router ([32, 2880])
│   │   │   │   └── experts (796,538,880 params)
│   │   │   ├── input_layernorm ([2880])
│   │   │   └── post_attention_layernorm ([2880])
│   │   ├── 5
│   │   │   ├── self_attn (64 params)
│   │   │   │   ├── q_proj ([4096, 2880])
│   │   │   │   ├── k_proj ([512, 2880])
│   │   │   │   ├── v_proj ([512, 2880])
│   │   │   │   └── o_proj ([2880, 4096])
│   │   │   ├── mlp
│   │   │   │   ├── router ([32, 2880])
│   │   │   │   └── experts (796,538,880 params)
│   │   │   ├── input_layernorm ([2880])
│   │   │   └── post_attention_layernorm ([2880])
│   │   ├── 6
│   │   │   ├── self_attn (64 params)
│   │   │   │   ├── q_proj ([4096, 2880])
│   │   │   │   ├── k_proj ([512, 2880])
│   │   │   │   ├── v_proj ([512, 2880])
│   │   │   │   └── o_proj ([2880, 4096])
│   │   │   ├── mlp
│   │   │   │   ├── router ([32, 2880])
│   │   │   │   └── experts (796,538,880 params)
│   │   │   ├── input_layernorm ([2880])
│   │   │   └── post_attention_layernorm ([2880])
│   │   ├── 7
│   │   │   ├── self_attn (64 params)
│   │   │   │   ├── q_proj ([4096, 2880])
│   │   │   │   ├── k_proj ([512, 2880])
│   │   │   │   ├── v_proj ([512, 2880])
│   │   │   │   └── o_proj ([2880, 4096])
│   │   │   ├── mlp
│   │   │   │   ├── router ([32, 2880])
│   │   │   │   └── experts (796,538,880 params)
│   │   │   ├── input_layernorm ([2880])
│   │   │   └── post_attention_layernorm ([2880])
│   │   ├── 8
│   │   │   ├── self_attn (64 params)
│   │   │   │   ├── q_proj ([4096, 2880])
│   │   │   │   ├── k_proj ([512, 2880])
│   │   │   │   ├── v_proj ([512, 2880])
│   │   │   │   └── o_proj ([2880, 4096])
│   │   │   ├── mlp
│   │   │   │   ├── router ([32, 2880])
│   │   │   │   └── experts (796,538,880 params)
│   │   │   ├── input_layernorm ([2880])
│   │   │   └── post_attention_layernorm ([2880])
│   │   ├── 9
│   │   │   ├── self_attn (64 params)
│   │   │   │   ├── q_proj ([4096, 2880])
│   │   │   │   ├── k_proj ([512, 2880])
│   │   │ 